In [1]:
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.utils import save_image

from tensorboardX import SummaryWriter

In [2]:
import sys
import os
import time
# sys.path.append('../src/oomugi/')
sys.path.append(os.path.abspath('../src/oomugi_add_spline/'))
import pprint
pprint.pprint(sys.path)

from model import UNet
from dataset import SSSDataset
from loss import DiscriminativeLoss

['',
 '/home/demo/.virtualenvs/pytorch-0.3.1/lib/python36.zip',
 '/home/demo/.virtualenvs/pytorch-0.3.1/lib/python3.6',
 '/home/demo/.virtualenvs/pytorch-0.3.1/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '/home/demo/.virtualenvs/pytorch-0.3.1/lib/python3.6/site-packages',
 '/home/demo/.virtualenvs/pytorch-0.3.1/lib/python3.6/site-packages/IPython/extensions',
 '/home/demo/.ipython',
 '/home/demo/document/ykato_git/omg_instance_segmentation/src/oomugi_add_spline']


In [3]:
n_sticks = 8
img_size = 256
batch_size = 1

In [4]:
# Model
model = UNet(img_size).cuda()

In [ ]:
# Dataset for train
train_dataset = SSSDataset(train=True, n_sticks=n_sticks, img_size=img_size)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=False, num_workers=0, pin_memory=True)

In [ ]:
# Loss Function
criterion_disc = DiscriminativeLoss(delta_var=0.5,
                                    delta_dist=1.5,
                                    norm=2,
                                    usegpu=True).cuda()
criterion_ce = nn.CrossEntropyLoss().cuda()
criterion_spline = nn.MSELoss().cuda()

In [ ]:
# Optimizer
parameters = model.parameters()
optimizer = optim.SGD(parameters, lr=0.01, momentum=0.9, weight_decay=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min',
                                                 factor=0.1,
                                                 patience=10,
                                                 verbose=True)

In [ ]:
# Train
save_dir = '../../result/omg_instance_segmentation/'
model_dir = Path('../../result/omg_instance_segmentation/model')

import socket
from datetime import datetime
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
log_dir = os.path.join(save_dir + 'runs', current_time + '_' + socket.gethostname())
writer = SummaryWriter(log_dir=log_dir)

best_loss = np.inf
start = time.time()
print("start")
for epoch in range(300):
    print(f'epoch : {epoch}')
    disc_losses = []
    ce_losses = []
    spline_losses = []
    for batched in train_dataloader:
        images, sem_labels, ins_labels, ins_num, spline_labels, spline_num = batched
        
        if ins_num[0] == 0:
            print('continue')
            continue
        
#         print(images.size())
#         print(sem_labels.size())
#         print(ins_labels.size())
#         print(spline_labels.size())
        
        nb ,nc, nh, nw = sem_labels.size()
        
#         print('images ', images.numpy().shape)
#         print('sem_labels ', sem_labels.numpy().shape)
#         print('ins_labels ', ins_labels.numpy().shape)
#         save_image(images,'debug_images.png', padding=10)
# #         save_image(torch.from_numpy(sem_labels.numpy()[:,0,:,:]).contiguous(),'debug_sem_labels.png', padding=10)
# #         save_image(ins_labels,'debug_ins_labels.png', padding=10)
#         print(type(sem_labels), sem_labels.size())
#         tmpTensor = sem_labels[:,0,:,:].contiguous()
#         save_image(tmpTensor.view(nb, 1, nh, nw),'debug_sem_labels.png', padding=10)
#         for i in range(8):
#             tmpTensor = ins_labels[:,i,:,:].contiguous()
#             save_image(tmpTensor.view(nb, 1, nh, nw),'debug_ins_labels{}.png'.format(i), padding=10)
        
        
        images = Variable(images).cuda()
        sem_labels = Variable(sem_labels).cuda()
        ins_labels = Variable(ins_labels).cuda()
        spline_labels = Variable(spline_labels).cuda()
        model.zero_grad()

        sem_predict, ins_predict, spline_predict = model(images)
        spline_predict = spline_predict.view((batch_size, 60, 8, 2))
        
#         print(sem_predict.size())
#         print(ins_predict.size())
#         print(spline_predict.size())
        
        loss = 0
        
#         print('sem_predict ', sem_predict.cpu().data.numpy().shape)
#         #save_image(sem_predict[:,0,:,:],'debug_sem_predict.png', padding=10)
#         tmpTensor = sem_predict.cpu().data
#         print(type(tmpTensor), tmpTensor.size())
#         tmpTensor = tmpTensor[:,0,:,:].contiguous()
#         save_image(tmpTensor.view(nb, 1, nh, nw),'debug_sem_predict.png', padding=10)
#         tmpTensor = ins_predict.cpu().data
#         print(type(tmpTensor), tmpTensor.size())
#         for i in range(16):
#             tmpTensor_a = tmpTensor[:,i,:,:].contiguous()
#             print('ins_predict', tmpTensor_a.view(nb, 1, nh, nw).size())
#             save_image(tmpTensor_a.view(nb, 1, nh, nw),'debug_ins_predict{}.png'.format(i), padding=10)

        # Discriminative Loss
#         disc_loss = criterion_disc(ins_predict,
#                                    ins_labels,
#                                    [n_sticks] * len(images))
#         print('ins_predict.size {}'.format(ins_predict.size()))
#         print('ins_labels.size {}'.format(ins_labels.size()))
#         print('ins_num.size {}'.format(ins_num.size()))


    
        disc_loss = criterion_disc(ins_predict, ins_labels, ins_num)
        loss += disc_loss
        disc_losses.append(disc_loss.cpu().data.numpy()[0])

        # Cross Entropy Loss
        _, sem_labels_ce = sem_labels.max(1)
        ce_loss = criterion_ce(sem_predict.permute(0, 2, 3, 1)\
                                   .contiguous().view(-1, 2),
                               sem_labels_ce.view(-1))
        loss += ce_loss
        ce_losses.append(ce_loss.cpu().data.numpy()[0])
        
        # Cross Entropy Loss
        # spline_labels : (1, 60*8*2)
        # spline_predict : (1,60*8*2)
        # print(spline_num)
        spline_predict = spline_predict.narrow(1, 0, spline_num[0])
        spline_labels = spline_labels.narrow(1, 0, spline_num[0])
#         print('predict', spline_predict.size())
#         print(spline_predict)
        
#         print('labels', spline_labels.size())
#         print(spline_labels)
        
        spline_loss = criterion_spline(spline_predict, spline_labels)
        
        loss += spline_loss
        spline_losses.append(spline_loss.cpu().data.numpy()[0])

        loss.backward()
        optimizer.step()
    disc_loss = np.mean(disc_losses)
    ce_loss = np.mean(ce_losses)
    spline_loss = np.mean(spline_losses)
    print(f'DiscriminativeLoss: {disc_loss:.4f}')
    print(f'CrossEntropyLoss: {ce_loss:.4f}')
    print(f'MSELoss: {spline_loss:.4f}')
    writer.add_scalar('train/disc_loss', disc_loss , epoch)
    writer.add_scalar('train/ce_loss', ce_loss , epoch)
    writer.add_scalar('train/spline_loss', spline_loss , epoch)
    scheduler.step(disc_loss)
#     if disc_loss < best_loss:
    if epoch % 30 == 0:
#         best_loss = disc_loss
#         print('Best Model!')
        modelname = 'model_epoch{}.pth'.format(str(epoch))
        torch.save(model.state_dict(), model_dir.joinpath(modelname))
        
elapsed_time = time.time() - start
print("elapsed_time:{0}".format(elapsed_time) + "[sec]")

start
epoch : 0
